**BOX API (Access Token) Authentication**
 - OAuth2 Authorization Code Grant
 - Need to issue CLIENT ID, SECRET, and allocate REDIRECT_URI

In [4]:
import webbrowser
import requests
from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import urlparse, parse_qs
from boxsdk import OAuth2, Client

CLIENT_ID = 'na2tcfu6nb8smov4l265dzq1a2v3i7av'
CLIENT_SECRET = 'fZ0UrVuPWtyDjlXe9xx309gDuJwxbRCw'
REDIRECT_URI = 'http://localhost:2129/callback'

auth_url = (
    f"https://account.box.com/api/oauth2/authorize"
    f"?response_type=code&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}"
)
webbrowser.open(auth_url)

class AuthHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        params = parse_qs(urlparse(self.path).query)
        code = params.get('code')
        if code:
            self.server.auth_code = code[0]
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"Authorization successful. You can close this tab.")
        else:
            self.send_response(400)
            self.end_headers()
            self.wfile.write(b"Missing authorization code.")

server = HTTPServer(('localhost', 2129), AuthHandler)
server.handle_request()
auth_code = server.auth_code

token_url = "https://api.box.com/oauth2/token"
data = {
    "grant_type": "authorization_code",
    "code": auth_code,
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "redirect_uri": REDIRECT_URI
}
res = requests.post(token_url, data=data)

if res.ok:
    token_data = res.json()
    print("Access Token:", token_data["access_token"])
    print("Refresh Token:", token_data["refresh_token"])
else:
    print("Error:", res.status_code, res.text)

oauth = OAuth2(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=token_data['access_token'],
    refresh_token=token_data['refresh_token'],
    store_tokens=lambda access, refresh: print("Token updated. Store securely.")
)
client = Client(oauth)


127.0.0.1 - - [05/Jun/2025 16:03:00] "GET /callback?state=&code=wv4Ys4icsfDWgUMa6p0Achqyy1yDOI4X HTTP/1.1" 200 -


Access Token: LedOGajhJSfo4JBZt0zLHXqelM4a4KxW
Refresh Token: 3CMuwKR4Fm5itwjFsbs0J9UYah7ulYw7I6sPSgLXF0bcecQOjOAe1FSiJZg0Gx90


**Loading Annotator - Imagery list(CapeTown_ImageIDs.xlsx)**
- Using BOX API

In [5]:
search_results = client.search().query('CapeTown_ImageIDs.xlsx')
for item in search_results:
    print(f"Found item: {item.name} (type: {item.type}, id: {item.id})")

Found item: CapeTown_ImageIDs.xlsx (type: file, id: 1665309472544)
Found item: CapeTown_ImageIDs.xlsx (type: file, id: 1665301575979)
Found item: capetown_2025_running_cost_5_01_lp.leap (type: file, id: 1850900416120)
Found item: CapeTown_1204 (type: folder, id: 296904851651)
Found item: capetown_2025_running_cost_4_11_LP.leap (type: file, id: 1831526175957)
Found item: capetown_2025_running_cost_4_2_LP.leap (type: file, id: 1822473916036)
Found item: capetown_2025_running_cost_3_25.leap (type: file, id: 1813568039579)
Found item: capetown_2025_running_cost_3_26_LP.leap (type: file, id: 1815193916444)
Found item: capetown_2025_running_cost_3_28_LP.leap (type: file, id: 1816545627861)
Found item: capetown_2025_running_cost_3_27_LP.leap (type: file, id: 1816367383099)
Found item: capetown_2025_running_cost_3_25_LP.leap (type: file, id: 1813872699775)
Found item: capetown_2025_running_cost_3_19_lp.leap (type: file, id: 1808185197085)
Found item: capetown_2025_running_cost_3_16_lp.leap (ty

**Matching annotators and imagery files**
- Download 'CapeTown_ImageIDs.xlsx' files
- Only 271 Aerial Images have annotations

In [6]:
import pandas as pd
import os

# Create Download folder
download_dir = 'download'
os.makedirs(download_dir, exist_ok=True)

# Define file path
file_id = '1665309472544' # refering to the above result
file_name = 'CapeTown_ImageIDs.xlsx'
file_path = os.path.join(download_dir, file_name)

# Download the file from Box
box_file = client.file(file_id).get()
with open(file_path, 'wb') as f:
    box_file.download_to(f)

print(f"Downloaded {file_name}")
# Load the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)
annotation_only = df[df["Status"]=='Completed']
annotation_only = annotation_only[['Image ID', 'Annotator']]
annotation_only

Downloaded CapeTown_ImageIDs.xlsx


/opt/miniconda3/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Image ID,Annotator
56,W07A_1,Gary Alvarez Mejia
63,W07A_16,Abby Finkle
72,W07A_24,Vanshika Mittal
106,W07C_10,Biz Yoder
107,W07C_11,Biz Yoder
...,...,...
2720,W59C_10,Abby Finkle
2737,W59D_20,Abby Finkle
2761,W66D_10,Abby Finkle
2776,W66D_4,Abby Finkle


**(Optional) Downloading the aerial image** 
- Option1. from Duke BOX Using API
- path : download/images

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

def download_file(image_id):
    search_query = f"2023_RGB_8cm_{image_id}.tif"
    try:
        search_results = client.search().query(
            query=image_id,
            ancestor_folder_ids=[target_folder_id],
            type='file',
            content_types=['name']
        )

        for item in search_results:
            if item.name == search_query:
                file_name = item.name
                file_id = item.id
                save_path = os.path.join(image_download_dir, file_name)
                box_file = client.file(file_id).get()
                with open(save_path, 'wb') as f:
                    box_file.download_to(f)
                return f"Downloaded: {file_name}"

        return f"Not found: {search_query}"

    except Exception as e:
        return f"Error with {search_query}: {str(e)}"

image_download_dir = 'download/images'
os.makedirs(image_download_dir, exist_ok=True)

target_folder_id = 285669537499
image_ids = annotation_only["Image ID"].tolist()

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(tqdm(executor.map(download_file, image_ids), total=len(image_ids)))

for res in results:
    print(res)

**(Optional) Downloading the aerial image**  (Recommended)
- Option2. from Local repository
- path : download/images

In [ ]:
import os
import shutil
from tqdm import tqdm

image_download_dir = 'download/images'
os.makedirs(image_download_dir, exist_ok=True)

source_dir = '/Volumes/One Touch/Data_Climate/Aerial Imagery/AP2023_TIFFs_Bass' # Need to be updated by users
target_dir = image_download_dir
os.makedirs(target_dir, exist_ok=True)

for image_id in tqdm(annotation_only["Image ID"]):
    filename = f"2023_RGB_8cm_{image_id}.tif"
    src_path = os.path.join(source_dir, filename)
    dst_path = os.path.join(target_dir, filename)

    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
        
    else:
        print(f"Not found: {filename}")

**Loading CRS(Coordinate Reference System)**
- For CRS, we are using `ESRI` not, ESRP which is not compatible with Python. Thus, we need to import .tif file that contains the CRS we use (ESRI:)
- ESRI:102562 (CapeTown Area)

In [ ]:
#Load sample imagery(.tif) and Extract its CRS(ESRI:102562)
import rasterio

input_tif = "./download/images/2023_RGB_8cm_W07A_1.tif" # File Path
with rasterio.open(input_tif) as dataset:
    geotif_crs = dataset.crs
print("CRS(ESRI:102562) is ready")

def clean_annotation_layers(layer, geotif_crs):
    layer = layer.to_crs(geotif_crs)
    layer['area'] = layer.geometry.area # Unit : square meter
    layer.reset_index(drop=True, inplace=True)
    return layer
print("Area of each polygon has been calculated in square feet.")

CRS(ESRI:102562) is ready
Area of each polygon has been calculated in square feet.


**Creating Annotator Lists**

In [8]:
selected_annotators = annotation_only['Annotator'].unique()
# 'Gary Alvarez Mejia', 'Abby Finkle', 'Vanshika Mittal','Biz Yoder', 'Zeinab Mukhtar', 'Shehr Naz Ashraf', 'Ye Khaung Oo', 'Veena Shirsath', 'Brian Mulu Mutua', 'Halle Evans','Ummamah Shah', 'Fiona Bolte-Bradhurst']

filtered_completed_df = annotation_only[annotation_only['Annotator'].isin(selected_annotators)]

key_mapping = {
    'Biz Yoder': 'biz',
    'Brian Mulu Mutua': 'mutua',
    'Fiona Bolte-Bradhurst': 'fiona',
    'Ummamah Shah': 'shah',
    'Ye Khaung Oo': 'ye',
    'Vanshika Mittal': 'mittal',
    'Zeinab Mukhtar': 'mukhtar',
    'Shehr Naz Ashraf': 'shehr',
    'Halle Evans': 'evans',
    'Abby Finkle' : 'abby',
    'Veena Shirsath' :'veen',
    'Gary Alvarez Mejia':'mejia'
}

annotator_image_ids = filtered_completed_df.groupby('Annotator')['Image ID'].apply(list).to_dict()
annotator_image_ids = {key_mapping[old_key]: value for old_key, value in annotator_image_ids.items()}
print(annotator_image_ids)

{'abby': ['W07A_16', 'W59B_10', 'W59B_3', 'W59B_8', 'W59B_9', 'W59C_10', 'W59D_20', 'W66D_10', 'W66D_4', 'W66D_5'], 'biz': ['W07C_10', 'W07C_11', 'W07C_12', 'W07C_13', 'W07C_16', 'W07C_17', 'W07C_2', 'W07C_21', 'W07C_22', 'W07C_23', 'W07C_3', 'W07C_4', 'W07C_5', 'W07C_6', 'W07C_7', 'W07C_8', 'W07C_9', 'W07D_1', 'W07D_6', 'W08A_1', 'W08A_12', 'W08A_2', 'W08B_4', 'W08B_9', 'W12A_17', 'W12A_18', 'W12A_21', 'W12A_22', 'W12A_23', 'W12A_24', 'W12C_11', 'W12C_12', 'W12C_16', 'W12C_17', 'W12C_3', 'W12C_4', 'W12C_6', 'W13A_1', 'W13C_11', 'W13C_13', 'W13C_16', 'W13C_17', 'W13C_18', 'W13C_21', 'W13C_22', 'W13C_8', 'W14A_11', 'W14A_12', 'W14A_16', 'W14A_21', 'W16C_14', 'W16C_15', 'W16C_17', 'W16C_18', 'W16C_19', 'W16C_20', 'W16C_21', 'W16C_22', 'W16D_11', 'W16D_12', 'W16D_16', 'W16D_17', 'W16D_21', 'W16D_22', 'W16D_23', 'W16D_24', 'W16D_25', 'W16D_7'], 'mutua': ['W33A_10', 'W33A_18', 'W33A_9', 'W33B_16', 'W33B_19', 'W33B_2', 'W33C_10', 'W33C_24', 'W33C_7', 'W33C_8', 'W33C_9', 'W33D_10', 'W33D_15',

**Loading Shapefiles**
- Make sure each shapefiles are up-to-date
- path : download/shapefiles ** manually uploaded the required files **
- shp, shx, dbf, prj, cpg, qix, qml files are needed

In [9]:
shapefile_download_dir = 'download/shapefiles'
os.makedirs(shapefile_download_dir, exist_ok=True)

file_names = {
    'biz': 'yoder_annotations 0128_v2.shp',
    'mutua': 'mutua_annotations 1027.shp',
    'fiona': 'bolte.bradhurst_annotations_1.31.shp',
    'shah': 'Shah_6M.shp',
    'ye': 'Ye_annotations.shp',
    'mittal': 'Mittal_annotations.shp',
    'mukhtar': 'mukhtar_annotations_dec5.shp',
    'shehr': 'ashraf_annotations.shp',
    'evans': 'evans_annotations_layer.shp',
    'mejia': 'alvarezmejia_annotations.shp',
    'veen' : 'shirsath_annotations.shp'
}

**Checking the number of annotations**
- Count rows of each shape files(.shp extension) 
- The '2020' layers were deleted. They did not match the coordinate reference system (CRS: ESRI 102562). (Confirmed with Biz.)

In [ ]:
import os
import geopandas as gpd

# Set up directory
shapefile_download_dir = 'download/shapefiles'
os.makedirs(shapefile_download_dir, exist_ok=True)

# List of shapefiles by annotator
file_names = {
    'biz': 'yoder_annotations 0128_v2.shp',
    'mutua': 'mutua_annotations 1027.shp',
    'fiona': 'bolte.bradhurst_annotations_1.31.shp',
    'shah': 'Shah_6M.shp',
    'ye': 'Ye_annotations.shp',
    'mittal': 'Mittal_annotations.shp',
    'mukhtar': 'mukhtar_annotations.shp',
    'shehr': 'ashraf_annotations.shp',
    'evans': 'evans_annotations_layer.shp',
    'veen': 'shirsath_annotations.shp',
    'mejia': 'alvarezmejia_annotations.shp'
}

# To store row counts and 2020-layer counts
row_counts = {}
layer_2020_counts = {}

for annotator, filename in file_names.items():
    path = os.path.join(shapefile_download_dir, filename)
    
    try:
        annotations = gpd.read_file(path)
    except Exception as e:
        print(f"Failed to read {filename}: {e}")
        row_counts[annotator] = 0
        layer_2020_counts[annotator] = 0
        continue

    # Total row count
    row_counts[annotator] = annotations.shape[0]
    
    # Count '2020' in layer column if exists
    if 'layer' in annotations.columns:
        layer_str = annotations['layer'].astype(str)
        layer_2020_counts[annotator] = layer_str.str.contains('2020', case=False, na=False).sum()
    else:
        layer_2020_counts[annotator] = 0

# Final report
for annotator in file_names.keys():
    total = row_counts.get(annotator, 0)
    layer_2020 = layer_2020_counts.get(annotator, 0)
    print(f"{annotator}: {total} rows (layer contains '2020': {layer_2020})")


biz: 10870 rows (layer contains '2020': 8480)
mutua: 8999 rows (layer contains '2020': 8480)
fiona: 13070 rows (layer contains '2020': 8156)
shah: 9228 rows (layer contains '2020': 8480)
ye: 11111 rows (layer contains '2020': 8480)
mittal: 11042 rows (layer contains '2020': 8480)
mukhtar: 12327 rows (layer contains '2020': 8480)
shehr: 11691 rows (layer contains '2020': 8433)
evans: 1354 rows (layer contains '2020': 0)
veen: 434 rows (layer contains '2020': 0)
mejia: 8780 rows (layer contains '2020': 8480)


**Data Formatting & Cleaning**
- Delete the rows containing any values in 'path' or 'layer' columns(legacy data)
- Add columns to record the vertices of each aerial imagery 
- 22519 rows × 17 columns

In [11]:
### Drop rows where either path or layer has a value(e.g 2020 layers) ###
import geopandas as gpd

# Annotation processing
annotations_list = []

for annotator, filename in file_names.items():
    path = os.path.join(shapefile_download_dir, filename)
    annotations = gpd.read_file(path)
    
    annotations['annotator'] = annotator

    # Ensure 'path' and 'layer' columns exist
    for col in ['path', 'layer']:
        if col not in annotations.columns:
            annotations[col] = None

    # Drop rows where either path or layer has a value
    annotations = annotations[annotations['path'].isnull() & annotations['layer'].isnull()]

    # Then clean geometry
    annotations = clean_annotation_layers(annotations, geotif_crs)

    annotations_list.append(annotations)

# Concatenate and post-process
annotations = pd.concat(annotations_list, ignore_index=True)
annotations.reset_index(drop=True, inplace=True)

# Add ID and centroid coordinates
annotations['id'] = annotations.index
annotations['centroid'] = annotations.geometry.centroid
annotations['centroid_latitude'] = annotations.centroid.y
annotations['centroid_longitude'] = annotations.centroid.x
annotations.drop(columns=['centroid'], inplace=True)

# Remove null geometries
annotations = annotations[annotations['geometry'].notnull()]

# placeholder metadata columns
annotations[['image_name',
             'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude',
             'se_corner_of_image_latitude', 'se_corner_of_image_longitude']] = None

annotations


,id,layer,path,PV_heater,uncertflag,PV_pool,area,geometry,annotator,PV_Pool,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,None,None,NaN,NaN,NaN,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,NaN,-3.769763e+06,-19991.996891,None,None,None,None,None
1,1,None,None,NaN,NaN,1.0,22.464224,"POLYGON ((-7338.039 -3775460.804, -7330.294 -3...",biz,NaN,-3.775459e+06,-7333.463276,None,None,None,None,None
2,2,None,None,NaN,NaN,1.0,71.734771,"POLYGON ((-4418.027 -3775551.846, -4406.117 -3...",biz,NaN,-3.775556e+06,-4412.644364,None,None,None,None,None
3,3,None,None,NaN,NaN,1.0,16.578681,"POLYGON ((-9145.183 -3780476.685, -9142.692 -3...",biz,NaN,-3.780474e+06,-9142.853580,None,None,None,None,None
4,4,None,None,NaN,NaN,1.0,27.684287,"POLYGON ((-8922.826 -3780035.448, -8914.535 -3...",biz,NaN,-3.780033e+06,-8917.850802,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22952,22952,None,None,NaN,NaN,NaN,13.599248,"POLYGON ((-49023.481 -3758771.941, -49020.398 ...",mejia,NaN,-3.758774e+06,-49022.331892,None,None,None,None,None
22953,22953,None,None,NaN,NaN,NaN,4.845304,"POLYGON ((-49074.803 -3758789.825, -49069.39 -...",mejia,NaN,-3.758791e+06,-49072.211780,None,None,None,None,None
22954,22954,None,None,NaN,NaN,NaN,4.371103,"POLYGON ((-49075.145 -3758791.263, -49069.938 ...",mejia,NaN,-3.758792e+06,-49072.717572,None,None,None,None,None
22955,22955,None,None,NaN,NaN,NaN,4.169215,"POLYGON ((-49075.214 -3758792.771, -49070.075 ...",mejia,NaN,-3.758794e+06,-49072.796580,None,None,None,None,None


**Bounding Box extraction** 
- Use `rasterio` to read each aerial imagery and extract boundary information
- The 'CapeTown_ImageIDs.xlsx' file does not include the name of all reference imagery files name
- Therefore, read all the imagery files directly instead

In [ ]:
## Download from external HDD ##

import os
import rasterio

def get_image_border_coordinates(image_path):
    with rasterio.open(image_path) as src:
        return src.bounds

# Directory containing the .tif images
folder_path = "/Volumes/One Touch/Data_Climate/Aerial Imagery/AP2023_TIFFs_Bass" # Need to download files in local reposotiry
# Save file as txt
output_file = "image_bounds.txt"

# Dictionary to store processed image bounds
processed_images = {}

# Scan and process only valid .tif files
for filename in os.listdir(folder_path):
    # Skip macOS metadata files like '._filename.tif'
    if filename.startswith("._"):
        continue

    if filename.endswith(".tif"):
        image_name = os.path.splitext(filename)[0]
        image_path = os.path.join(folder_path, filename)

        if image_name not in processed_images:
            try:
                bounds = get_image_border_coordinates(image_path)
                processed_images[image_name] = bounds
            except rasterio.errors.RasterioIOError as e:
                print(f"Skipped {filename} due to read error: {e}")

# Format and print the results
formatted_bounds = []
for image_name, bounds in processed_images.items():
    formatted_bounds.append(
        f"{image_name}: left={bounds.left}, bottom={bounds.bottom}, right={bounds.right}, top={bounds.top}"
    )

for line in formatted_bounds:
    print(line)

# Save results to a text file
with open(output_file, "w") as f:
    for image_name, bounds in processed_images.items():
        line = f"{image_name}: left={bounds.left}, bottom={bounds.bottom}, right={bounds.right}, top={bounds.top}\n"
        f.write(line)

print(f"Finished processing. Results saved to {output_file}")

**Matching annotations with aerial imagery**
- Use cetroid_latitude & longitude to determine which image each annotations belongs to
- 22519 rows × 17 columns

In [ ]:
import pandas as pd

# === Step 0: Read image_bounds.txt file ===
def load_bounds_from_txt(txt_path):
    bounds_dict = {}
    with open(txt_path, "r") as f:
        for line in f:
            if line.strip() == "":
                continue
            try:
                # Example line: 2023_RGB_8cm_W17B_11: left=595000.0, bottom=4132000.0, right=596000.0, top=4133000.0
                image_name, rest = line.strip().split(":", 1)
                parts = rest.strip().split(",")
                bound_values = {p.split("=")[0].strip(): float(p.split("=")[1]) for p in parts}
                bounds_dict[image_name] = bound_values
            except Exception as e:
                print(f"Error parsing line: {line.strip()} | Error: {e}")
    return bounds_dict

# Load bounds from file
txt_file = "image_bounds.txt"  # Replace with your actual path if needed
annotator_border_coordinates = load_bounds_from_txt(txt_file)

# Point-in-Bounds function
def is_point_within_bounds(lat, lon, bounds):
    return bounds['left'] <= lon <= bounds['right'] and bounds['bottom'] <= lat <= bounds['top']

# Annotate polygons with image name and corner bounds
# Assume `annotations` is a GeoDataFrame or DataFrame that includes centroid_latitude and centroid_longitude columns
for idx, row in annotations.iterrows():
    centroid_lat = row['centroid_latitude']
    centroid_lon = row['centroid_longitude']
    
    for image_name, bounds in annotator_border_coordinates.items():
        if is_point_within_bounds(centroid_lat, centroid_lon, bounds):
            annotations.loc[idx, 'image_name'] = image_name
            annotations.loc[idx, 'nw_corner_of_image_latitude'] = bounds['top']
            annotations.loc[idx, 'nw_corner_of_image_longitude'] = bounds['left']
            annotations.loc[idx, 'se_corner_of_image_latitude'] = bounds['bottom']
            annotations.loc[idx, 'se_corner_of_image_longitude'] = bounds['right']
            break

annotations

,id,layer,path,PV_heater,uncertflag,PV_pool,area,geometry,annotator,PV_Pool,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,None,None,NaN,NaN,NaN,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,NaN,-3.769763e+06,-19991.996891,2023_RGB_8cm_W16C_21,-3769000.0,-20000.0,-3770000.0,-19000.0
1,1,None,None,NaN,NaN,1.0,22.464224,"POLYGON ((-7338.039 -3775460.804, -7330.294 -3...",biz,NaN,-3.775459e+06,-7333.463276,2023_RGB_8cm_W07C_3,-3775000.0,-8000.0,-3776000.0,-7000.0
2,2,None,None,NaN,NaN,1.0,71.734771,"POLYGON ((-4418.027 -3775551.846, -4406.117 -3...",biz,NaN,-3.775556e+06,-4412.644364,2023_RGB_8cm_W07D_1,-3775000.0,-5000.0,-3776000.0,-4000.0
3,3,None,None,NaN,NaN,1.0,16.578681,"POLYGON ((-9145.183 -3780476.685, -9142.692 -3...",biz,NaN,-3.780474e+06,-9142.853580,2023_RGB_8cm_W08A_1,-3780000.0,-10000.0,-3781000.0,-9000.0
4,4,None,None,NaN,NaN,1.0,27.684287,"POLYGON ((-8922.826 -3780035.448, -8914.535 -3...",biz,NaN,-3.780033e+06,-8917.850802,2023_RGB_8cm_W08A_2,-3780000.0,-9000.0,-3781000.0,-8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22952,22952,None,None,NaN,NaN,NaN,13.599248,"POLYGON ((-49023.481 -3758771.941, -49020.398 ...",mejia,NaN,-3.758774e+06,-49022.331892,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
22953,22953,None,None,NaN,NaN,NaN,4.845304,"POLYGON ((-49074.803 -3758789.825, -49069.39 -...",mejia,NaN,-3.758791e+06,-49072.211780,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
22954,22954,None,None,NaN,NaN,NaN,4.371103,"POLYGON ((-49075.145 -3758791.263, -49069.938 ...",mejia,NaN,-3.758792e+06,-49072.717572,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
22955,22955,None,None,NaN,NaN,NaN,4.169215,"POLYGON ((-49075.214 -3758792.771, -49070.075 ...",mejia,NaN,-3.758794e+06,-49072.796580,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0


**Addational Data Cleaning (8steps) -> 19,735 unique annotations**
- Drop duplicates corresponding to 'geometry'
- Merge PV_Pool into PV_pool.
- Ensure binary values '1' or 'Nan' for PV related columns.
- Create a 'PV_normal' column if all other PV related colums are NaN.
- Reorder columns for clarity
- Drop unnecessary columns (layer, path = '2020 layers').
- Reindex 'id' for consistency.


- !!! Around 1.9K annotations are missing imagery files, 'CapeTown_ImageIDs.xlsx' does not cover all required imagey files. We found additional 23 imagery files that to match all annotations !!!
    - Missing 23 images:2023_RGB_8cm_W07A_11, 2023_RGB_8cm_W14A_1,2023_RGB_8cm_W16C_23,2023_RGB_8cm_W16C_25,2023_RGB_8cm_W17B_18, 2023_RGB_8cm_W17B_8,2023_RGB_8cm_W17B_9,2023_RGB_8cm_W24A_16,2023_RGB_8cm_W24A_19,2023_RGB_8cm_W25C_14,2023_RGB_8cm_W25C_15,2023_RGB_8cm_W25C_16,2023_RGB_8cm_W36A_9,2023_RGB_8cm_W36B_15,2023_RGB_8cm_W36B_25,2023_RGB_8cm_W36B_3,2023_RGB_8cm_W46C_7,2023_RGB_8cm_W46C_8,2023_RGB_8cm_W47A_6,2023_RGB_8cm_W55A_20,2023_RGB_8cm_W55C_18,2023_RGB_8cm_W57C_13,2023_RGB_8cm_W58B_17

In [ ]:
import pandas as pd
import numpy as np

# Drop duplicate polygons based on 'geometry'
annotations_cleaned = annotations.drop_duplicates(subset='geometry', keep='first')

# # Merge 'PV_Pool' into 'PV_pool' and remove 'PV_Pool'(Option1)
# annotations_cleaned['PV_pool'] = annotations_cleaned['PV_pool'].fillna(annotations_cleaned['PV_Pool'])
# annotations_cleaned.drop(columns=['PV_Pool'], inplace=True)

# # Merge 'PV_Pool' into 'PV_pool' and remove 'PV_Pool'(Option2)
if 'PV_pool' in annotations_cleaned.columns and 'PV_Pool' in annotations_cleaned.columns:
    annotations_cleaned['PV_pool'] = annotations_cleaned['PV_pool'].combine_first(annotations_cleaned['PV_Pool'])
    annotations_cleaned.drop(columns=['PV_Pool'], inplace=True)

# Ensure binary values (1 or nan) in 
cols_to_check = ['PV_heater', 'uncertflag', 'PV_pool']
for col in cols_to_check:
    annotations_cleaned[col] = annotations_cleaned[col].apply(
        lambda x: 1 if pd.notnull(x) and x > 1 else (np.nan if x == 0 else x)
    )

# Create 'PV_normal' if 'PV_heater', 'PV_pool', and 'uncertflag' are all NaN
condition = annotations_cleaned[['PV_heater', 'PV_pool', 'uncertflag']].isna().all(axis=1)
annotations_cleaned['PV_normal'] = np.where(condition, 1, np.nan)


desired_order = ['id', 'PV_normal', 'PV_heater', 'PV_pool', 'uncertflag']


remaining_cols = [col for col in annotations_cleaned.columns if col not in desired_order]

# Apply reordering
annotations_cleaned = annotations_cleaned[desired_order + remaining_cols]

# Drop unnecessary columns
annotations_cleaned.drop(columns=['layer', 'path'], inplace=True, errors='ignore')

# Reset and renumber 'id'
annotations_cleaned.reset_index(drop=True, inplace=True)
annotations_cleaned['id'] = annotations_cleaned.index + 1

# Save for check
annotations_cleaned.to_csv("annotations_cleaned.csv", index=False)

# Output
annotations_cleaned

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/x0/gkxw33gj58n2ptpf02t8_ncr0000gn/T/ipykernel_54722/3798471344.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations_cleaned.drop(columns=['PV_Pool'], inplace=True)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,id,PV_normal,PV_heater,PV_pool,uncertflag,area,geometry,annotator,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,1,1.0,NaN,NaN,NaN,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,-3.769763e+06,-19991.996891,2023_RGB_8cm_W16C_21,-3769000.0,-20000.0,-3770000.0,-19000.0
1,2,NaN,NaN,1.0,NaN,22.464224,"POLYGON ((-7338.039 -3775460.804, -7330.294 -3...",biz,-3.775459e+06,-7333.463276,2023_RGB_8cm_W07C_3,-3775000.0,-8000.0,-3776000.0,-7000.0
2,3,NaN,NaN,1.0,NaN,71.734771,"POLYGON ((-4418.027 -3775551.846, -4406.117 -3...",biz,-3.775556e+06,-4412.644364,2023_RGB_8cm_W07D_1,-3775000.0,-5000.0,-3776000.0,-4000.0
3,4,NaN,NaN,1.0,NaN,16.578681,"POLYGON ((-9145.183 -3780476.685, -9142.692 -3...",biz,-3.780474e+06,-9142.853580,2023_RGB_8cm_W08A_1,-3780000.0,-10000.0,-3781000.0,-9000.0
4,5,NaN,NaN,1.0,NaN,27.684287,"POLYGON ((-8922.826 -3780035.448, -8914.535 -3...",biz,-3.780033e+06,-8917.850802,2023_RGB_8cm_W08A_2,-3780000.0,-9000.0,-3781000.0,-8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,19731,1.0,NaN,NaN,NaN,13.599248,"POLYGON ((-49023.481 -3758771.941, -49020.398 ...",mejia,-3.758774e+06,-49022.331892,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
19731,19732,1.0,NaN,NaN,NaN,4.845304,"POLYGON ((-49074.803 -3758789.825, -49069.39 -...",mejia,-3.758791e+06,-49072.211780,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
19732,19733,1.0,NaN,NaN,NaN,4.371103,"POLYGON ((-49075.145 -3758791.263, -49069.938 ...",mejia,-3.758792e+06,-49072.717572,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0
19733,19734,1.0,NaN,NaN,NaN,4.169215,"POLYGON ((-49075.214 -3758792.771, -49070.075 ...",mejia,-3.758794e+06,-49072.796580,2023_RGB_8cm_W45C_16,-3758000.0,-50000.0,-3759000.0,-49000.0


**Save Dataframe as GPKG**
- Shapefiles have a limitation: field (column) names must be fewer than 10 characters. Therefore, GPKG is preferrered here.
- Note: ESRI 102562 is equal to EPSG:4326 (WGS84) https://spatialreference.org/ref/esri/102562/wkt.html.
- The full WKT format of ESRI 102562 is privided in the following code block.

In [ ]:
"""
ESRI:102562
PROJCS["Hartebeesthoek94_Lo19_(E-N)",
    GEOGCS["Hartebeesthoek94",
        DATUM["Hartebeesthoek94",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6148"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4148"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",19],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["ESRI","102562"]]
"""

import rasterio
import geopandas as gpd
from pyproj import CRS

# Import ESRI(102562) embedded in sample tif image
input_tif = "./download/images/2023_RGB_8cm_W07A_1.tif"
with rasterio.open(input_tif) as dataset:
    geotif_wkt = dataset.crs.to_wkt()  # Get CRS as WKT string
print(geotif_wkt)


crs = CRS.from_wkt(geotif_wkt)
annotations_cleaned.to_crs(crs, inplace=True) ## to_crs, set_crs ?
annotations_cleaned.to_file("final_annotations.gpkg", layer='annotations', driver="GPKG")

print("GPKG format annotation file created")

PROJCS["GCS_WGS_1984",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.25722356049,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",19],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
GPKG format annotation file created
